In [2]:
from collections import OrderedDict
import geocoder
import pickle
import operator


In [32]:
def getGeoCoding(name):
    g = geocoder.google(name)
    return g.latlng

def getCountryGeo(locations,name):
    if name in ["''","'.'",0]:
        return locations
    else:
        
        if locations.get(name,''):
            if locations[name]==None:
                locations[name]=getGeoCoding(name)
        else:
            locations[name]=getGeoCoding(name)
        return locations
    
def plotNetworkGraph(waveMatrix, peelMatrix,locations,nodeNames, filename, thresholds, nodeSize):
    print(waveMatrix)
    skip=[]

    if int(filename.split("/")[-1])%4==0:
        resolution='l'
    else:
        resolution='l'
    calCorrDistribution(waveMatrix,nodeNames,filename)
    pickle.dump(waveMatrix,open(filename.replace('/','_'),"wb"))
    for thres in thresholds:
        Bmap_Amplitude = Basemap(projection='merc',llcrnrlon=lower_left_lon,llcrnrlat=lower_left_lat,urcrnrlon=upper_right_lon,urcrnrlat=upper_right_lat,lat_ts=0,resolution=resolution,suppress_ticks=True)

        G_Amplitude = nx.Graph()

        for name in nodeNames:
            try:
                longitude=locations[int(name)][0]
                latitude=locations[int(name)][1]
            except:
                longitude=locations[name][0]
                latitude=locations[name][1]
            #print(latitude,longitude)
            if(validateLocation(latitude,longitude)):
                x,y=Bmap_Amplitude(latitude,longitude)
                #print(x,y)
                G_Amplitude.add_node(name,pos=(x,y))
            else:
                skip.append(name)

        for i in range(len(peelMatrix)):
            for j in range(i+1,len(peelMatrix)):
                    node1 = nodeNames[i]
                    node2 = nodeNames[j]
                    wC = waveMatrix[i][j]
                    #pC = peelMatrix[i][j]


                    highWave = (wC > thres)

                    if(highWave):
                        if node1 not in skip and node2 not in skip:
                            G_Amplitude.add_edge(node1, node2, weight = wC)

                    else:
                        continue

        plt.close()
        #plt.figure(figsize = (8,5), dpi = 300)
        #gs = gridspec.GridSpec(3,4)
        plt.axis('off')

        #plt.subplot(gs[0,0:2])
        year=filename.split('/')[-1]
        plt.title(year,fontsize = 14)
        edgewidth_Amplitude = [ d['weight'] for (u,v,d) in G_Amplitude.edges(data=True)]
        pos=nx.get_node_attributes(G_Amplitude,'pos')
        nx.draw_networkx_nodes(G_Amplitude,pos, node_color = 'r', node_size = nodeSize, alpha = 0.7)
        nx.draw_networkx_edges(G_Amplitude, pos, edge_color = edgewidth_Amplitude, width=edgewidth_Amplitude, alpha = edgewidth_Amplitude)
        Bmap_Amplitude.drawcountries()
        Bmap_Amplitude.drawstates()
        try:
            Bmap_Amplitude.bluemarble()
        except:
            try:
                Bmap_Amplitude.drawlsmask(land_color='coral',ocean_color='aqua',lakes=True)
            except:
                pass
        plt.axis('off')

        print(filename)
        lp=filename.split('/')[:-2]
        year=filename.split('/')[-2:]
        plt.savefig("/".join(lp)+ "/results_"+str(thres)+"{}.png".format(year), dpi = 300,bbox_inches='tight')



In [41]:
file="pipeFormat.csv"
f=open(file,"r")
print(f.readline())
data=OrderedDict()
nagid={}
targets={}
for l in f:
    row=l.split("|")
    year=int(row[3])
    target=row[8]
    support=row[38]
    target_num=row[9]
    nagids=row[16:21]
    try:
        if support not in ['.','','0']:
            nagids=[int(i[0]) for i in nagids]
            key=support+"|"+target
            for i in range(0,len(nagids)):
                if nagids[i]==1:
                    if key not in nagid:
                        nagid[key]=set()
                    nagid[key].add(i)    
    except:
        pass
    
    
    if year not in data:
        data[year]=[]
    if target not in targets:
        targets[target]=int(target_num)
    data[year].append((support,target,))


triadid|UCDPdyadid|conflictid|year|yr_active|foundyr|startdate1|startdate2|target|tarnum_cow|tarnum_gw|nag_name|nagcode_1|nagcode_2|incomp|terr|nagid_1|nagid_2|nagid_3|nagid_4|nagid_5|nageth|nagrel|nagleft|nagright|nagdem|nagauth|nagdict|nagmil|nagtheo|nagobj_1|nagobj_2|nagobj_3|nagobj_4|nagobj_5|nagobj_6|polpardummy|partyname|supporter|supnum_gw|supnum_COW|statesup|s_safemem|s_precision_sm|s_safelead|s_precision_sl|s_headquar|s_precision_head|s_traincamp|s_precision_tc|s_training|s_precision_train|s_weaponlog|s_precision_wl|s_finaid|s_precision_finaid|s_transport|s_precision_tp|s_troop|s_precision_tr|s_other|s_precision_oth|defacto|ds_safemem|ds_precision_sm|ds_safelead|ds_precision_sl|ds_headquar|ds_precision_head|ds_traincamp|ds_precision_tc|ds_training|ds_precision_train|ds_weaponlog|ds_precision_wl|ds_finaid|ds_precision_finaid|ds_transport|ds_precision_tp|ds_other|ds_precision_oth|domsup|domsup_p



In [48]:
temp=set()
for keys in nagid:
    if len(nagid[keys])>1:
        try:
            nagid[keys].remove(0)
        except:
            pass
    temp.add(tuple(nagid[keys]))
temp

{(0,),
 (1,),
 (1, 2),
 (1, 2, 3),
 (1, 2, 3, 4),
 (1, 3),
 (1, 4),
 (2,),
 (2, 4),
 (3,),
 (3, 4),
 (4,)}

In [55]:
pickle.dump(nagidMap,open("nagidMap.pickle","wb"))

In [53]:
nagidMap={}
count=0
for keys in temp:
    nagidMap[keys]=count
    count+=1

In [11]:
a=[0,1,0,1,1,0,1]
a.count(1)

4

In [56]:
nadidDefination=OrderedDict()
nadidDefination[(0,)]="None"
nadidDefination[(1,)]="Ethno-nationalist"
nadidDefination[(2,)]="Religious"
nadidDefination[(3,)]="Leftist"
nadidDefination[(4,)]="Other"
nadidDefination[(1, 2)]="Ethno-nationalist + Religious"
nadidDefination[(1, 3)]="Ethno-nationalist + Leftist"
nadidDefination[(1, 4)]="Ethno-nationalist + Other"
nadidDefination[(2, 4)]="Religious + Leftist"
nadidDefination[(3, 4)]="Leftist + Other"
nadidDefination[(1, 2, 3)]="Ethno-nationalist + Religious + Leftist"
nadidDefination[(1, 2, 3, 4)]="Ethno-nationalist + Religious + Leftist + Others"


pickle.dump(nadidDefination,open("nadidDefination.pickle","wb"))  

In [6]:
# Targets Continent
# 2 - 95 NA   
#100 - 165 SA  
# 200 - 360 Europe 
#165 -173 Asia -1 
# 404 - 625 Africa 
# 630 - 698 MEast
# 630 - 910 Asia 
# ME
targetWeights={}
for target in targets:
    if 2<=targets[target]<=95:
        targetWeights[target]=0.05
    elif 100<=targets[target]<=165:
        targetWeights[target]=0.25
    elif 200<=targets[target]<=360:
        targetWeights[target]=0.38
    elif 404<=targets[target]<=625:
        targetWeights[target]=0.55
    elif 630<=targets[target]<=698:
        targetWeights[target]=0.75
    else:
        targetWeights[target]=0.95
        
        

In [7]:

pickle.dump(targetWeights,open("targetWeights.pickle","wb"))  

In [33]:
try:
    locations=pickle.load(open("locations.pickle","rb"))
except:
    locations={}
for year in data:
    for country in data[year]:
        locations=getCountryGeo(locations,country[0])
        locations=getCountryGeo(locations,country[1])
pickle.dump(locations,open("locations.pickle","wb"))      

ERROR:geocoder.base:Status code 400 from https://maps.googleapis.com/maps/api/geocode/json: ERROR - 400 Client Error: Bad Request for url: https://maps.googleapis.com/maps/api/geocode/json?address=&bounds=&components=&region=&language=
ERROR:geocoder.base:Status code 400 from https://maps.googleapis.com/maps/api/geocode/json: ERROR - 400 Client Error: Bad Request for url: https://maps.googleapis.com/maps/api/geocode/json?address=&bounds=&components=&region=&language=
ERROR:geocoder.base:Status code 400 from https://maps.googleapis.com/maps/api/geocode/json: ERROR - 400 Client Error: Bad Request for url: https://maps.googleapis.com/maps/api/geocode/json?address=&bounds=&components=&region=&language=
ERROR:geocoder.base:Status code 400 from https://maps.googleapis.com/maps/api/geocode/json: ERROR - 400 Client Error: Bad Request for url: https://maps.googleapis.com/maps/api/geocode/json?address=&bounds=&components=&region=&language=
ERROR:geocoder.base:Status code 400 from https://maps.go

ERROR:geocoder.base:Status code 400 from https://maps.googleapis.com/maps/api/geocode/json: ERROR - 400 Client Error: Bad Request for url: https://maps.googleapis.com/maps/api/geocode/json?address=&bounds=&components=&region=&language=
ERROR:geocoder.base:Status code 400 from https://maps.googleapis.com/maps/api/geocode/json: ERROR - 400 Client Error: Bad Request for url: https://maps.googleapis.com/maps/api/geocode/json?address=&bounds=&components=&region=&language=
ERROR:geocoder.base:Status code 400 from https://maps.googleapis.com/maps/api/geocode/json: ERROR - 400 Client Error: Bad Request for url: https://maps.googleapis.com/maps/api/geocode/json?address=&bounds=&components=&region=&language=
ERROR:geocoder.base:Status code 400 from https://maps.googleapis.com/maps/api/geocode/json: ERROR - 400 Client Error: Bad Request for url: https://maps.googleapis.com/maps/api/geocode/json?address=&bounds=&components=&region=&language=
ERROR:geocoder.base:Status code 400 from https://maps.go

In [57]:
nagid=pickle.load(open("nagid.pickle","rb")) 
nagid

{'Afghanistan|Algeria': {2},
 'Afghanistan|China': {1, 2},
 'Afghanistan|India': {1},
 'Afghanistan|Iraq': {1, 2},
 'Afghanistan|Myanmar (Burma)': {1, 2},
 'Afghanistan|Nigeria': {2},
 'Afghanistan|Pakistan': {1, 2},
 'Afghanistan|Philippines': {1, 2},
 'Afghanistan|Tajikistan': {2, 4},
 'Albania|Serbia (Yugoslavia)': {1},
 'Algeria|Chad': {1},
 'Algeria|Israel': {1, 3},
 'Algeria|Mali': {1},
 'Algeria|Mauritania': {1},
 'Algeria|Morocco': {1},
 'Algeria|Nigeria': {2},
 'Algeria|Spain': {1, 3},
 'Algeria|Tunisia': {0},
 'Angola|Congo': {0},
 'Angola|DR Congo (Zaire)': {1},
 'Angola|South Africa': {1, 3},
 'Argentina|Nicaragua': {4},
 'Armenia|Azerbaijan': {1},
 'Australia|Indonesia': {1},
 'Australia|Sri Lanka': {1},
 'Austria|Iraq': {2},
 'Azerbaijan|Iran': {1, 3},
 'Bahrain|United Kingdom': {1},
 'Bangladesh|India': {1, 2, 3},
 'Bangladesh|Myanmar (Burma)': {1, 2},
 'Belgium|France': {1},
 'Belgium|Rwanda': {1},
 'Belgium|Turkey': {1, 3},
 'Benin|Central African Republic': {1},
 'Bhu

In [2]:
locations=pickle.load(open("locations.pickle","rb"))

In [ ]:
for nm in locations:
    if nm[0]

In [17]:
getGeoCoding('Bhutan')

[27.514162, 90.433601]

In [46]:
ab=[]
for nm in locations:
    if locations[nm]==None:
        ab.append(nm)
print(ab)

['', '.', '0']


In [7]:
pickle.dump(locations,open("locations.pickle","wb")) 

In [4]:
locations['Russia (Soviet Union)']=(61.52,105.31)
locations['Russia(Soviet Union)']=(61.52,105.31)
locations['The United States']=[37.09024, -95.712891]
locations['My+I104anmar (Burma)']=(21.91,95.95)
locations['German Democratic Republic']=(52.52,13.40)
locations['Somali']=(5.15,46.19)
locations['Benin']=(9.30,2.31)

In [5]:
del locations['sThe United States']

In [48]:
del locations[''] 
del locations['.'] 
del locations['0'] 

In [38]:
nodenames=[]
for nm in locations:
    if nm not in ['','.','0']:
        nodenames.append(nm)
nodenames.sort()
pickle.dump(nodenames,open("nodenames.pickle","wb"))      


In [39]:
locations

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belgium',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Bosnia-Herzegovina',
 'Botswana',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cambodia',
 'Cambodia (Kampuchea)',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo, Democratic Republic of (Zaire)',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'DR Congo (Zaire)',
 'Democratic Republic of Congo',
 'Democratic Republic of Congo (Zaire)',
 'Denmark',
 'Djibouti',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'El Savador',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'German Democratic Republic',
 'German Federal Republic',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Haiti',
 'Honduras',
 'India',
 'Indonesia',
 'I

In [1]:
ap=[0,1,2,3]
ap=[-1]+[0,1,2,3]

In [2]:
ap


[-1, 0, 1, 2, 3]